In [8]:
import itertools
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.metrics import classification_report

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d iamsouravbanerjee/animal-image-dataset-90-different-animals

 97% 634M/656M [00:07<00:00, 71.4MB/s]
100% 656M/656M [00:07<00:00, 93.0MB/s]


In [5]:
with zipfile.ZipFile('/content/animal-image-dataset-90-different-animals.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [9]:
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [15]:
dataset = ImageFolder(root="/content/animals/animals", transform=transform)
class_names = dataset.classes

In [16]:
train_size = int(0.8 * len(dataset))
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [17]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16 * 54 * 54, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(class_names))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 54 * 54)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [18]:
model = ConvolutionalNetwork()

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())


In [20]:
print(classification_report(y_true, y_pred, target_names=class_names))

                precision    recall  f1-score   support

      antelope       0.00      0.00      0.00        13
        badger       0.00      0.00      0.00        15
           bat       0.00      0.00      0.00        11
          bear       0.00      0.00      0.00        11
           bee       0.00      0.00      0.00        13
        beetle       0.00      0.00      0.00        13
         bison       0.00      0.00      0.00         9
          boar       0.00      0.00      0.00        15
     butterfly       0.00      0.00      0.00         6
           cat       0.00      0.00      0.00        13
   caterpillar       0.00      0.00      0.00        12
    chimpanzee       0.00      0.00      0.00        20
     cockroach       0.00      0.00      0.00        11
           cow       0.01      0.43      0.02         7
        coyote       0.00      0.00      0.00        17
          crab       0.00      0.00      0.00        13
          crow       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
